In [1]:
import requests
import pandas as pd
import numpy as np
import string
import warnings
import matplotlib.pyplot as pltimport 
import pyLDAvis.sklearn

from pprint import pprint
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
from tqdm import tqdm
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig

warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.7/site-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatvalue=lambda value: "")[1:-1]
/usr/local/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [2]:
# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
#!python3 -m spacy download en_core_web_sm

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', -1)

### Build the Google News Scraper using BeautifulSoup
---

In [4]:
def covid19_news_scraper(search_query):
    """
    Pass in a client name or search query and returns last 100 headlines associating the client with Covid-19   
    """
    # Use this URL for Australian centric data
    news_url = "https://news.google.com.au/rss/search?q={"+str(search_query)+"%coronavirus}"
    Client = urlopen(news_url)
    xml_page = Client.read()
    Client.close()
    # Beautiful Soup Library is the bomb
    soup_page = soup(xml_page,"xml")
    news_list = soup_page.findAll("item")
    # Two separate lists for News Title and Publication Date
    l1 = []
    l2 = []
    for news in news_list:
        # Append to a list
        l1.append(news.title.text)
        l2.append(news.pubDate.text)
        # Zip the two together
        l_tup = list(zip(l1, l2))
    # Save this to a DataFrame
    df = pd.DataFrame(l_tup, columns=['Title', 'Date'])
    # Select Date of Headline
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    # Split the Title into Headline and Source columns and then drop the 'Title' column
    df[['Headline','Source']] = df['Title'].str.rsplit("-",1,expand=True)
    df.drop('Title', axis=1, inplace=True)
    return df

nab_df = covid19_news_scraper('NAB')
print(nab_df.shape)
nab_df.head()

(100, 3)


,Date,Headline,Source
0,2020-04-20,"ASX drops 2.5pc as oil prices collapse, while NAB flags $1.1b hit to earnings",ABC News
1,2020-04-20,NAB flags $1.14bn triple hit to H1 result,The West Australian
2,2020-03-24,NAB worker sacked over false coronavirus test,The New Daily
3,2020-04-17,NAB's McEwan vows fight to save jobs in bank restructure,Sydney Morning Herald
4,2020-04-20,Coronavirus credit card cuts: Which banks have slashed rates?,Mozo.com.au


In [5]:
cba_df = covid19_news_scraper('CBA')
print(cba_df.shape)
cba_df.head()

(100, 3)


,Date,Headline,Source
0,2020-04-14,Hoarding boom is over: CBA says shoppers have closed their wallets,The Age
1,2020-04-16,"Sydney, Melbourne house prices facing 10 per cent fall: CBA",Brisbane Times
2,2020-03-31,Coronavirus: CBA report reveals what we’re buying during pandemic,NEWS.com.au
3,2020-03-23,"Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall",Canstar
4,2020-04-01,CBA increases coronavirus loan support,7NEWS.com.au


In [6]:
anz_df = covid19_news_scraper('ANZ')
print(anz_df.shape)
anz_df.head()

(100, 3)


,Date,Headline,Source
0,2020-03-24,ANZ signals further relief for customers during crisis,6PR
1,2020-04-04,'Australia won't look the same': ANZ's Elliott warns coronavirus impact will be generational,The Age
2,2020-04-19,'Everybody is paying': ANZ Bank chairman David Gonski,Brisbane Times
3,2020-03-24,Big bank coronavirus support: How to defer your mortgage repayments,Mozo.com.au
4,2020-04-19,Gonski flags hit to dividend at ANZ,The Australian


In [7]:
wbc_df = covid19_news_scraper('Westpac')
print(wbc_df.shape)
wbc_df.head()

(100, 3)


,Date,Headline,Source
0,2020-04-16,Riskier funds have been exposed by coronavirus: Westpac,The Australian Financial Review
1,2020-04-14,"Westpac flags $1.4b hit to earnings, braces for coronavirus impact",Sydney Morning Herald
2,2020-04-15,Westpac survey reveals coronavirus caused consumer confidence to fall,NEWS.com.au
3,2020-04-20,"ASX drops 2.5pc as oil prices collapse, while NAB flags $1.1b hit to earnings",ABC News
4,2020-03-23,"Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall",Canstar


### Sentiment Analyis on Text
---

In [8]:
def covid19_news_scraper(search_query):
    """
    Pass in a client name or search query and returns last 100 headlines associating the client with Covid-19   
    """
    # Use this URL for Australian centric data
    news_url = "https://news.google.com.au/rss/search?q={"+str(search_query)+"%coronavirus}"
    Client = urlopen(news_url)
    xml_page = Client.read()
    Client.close()
    # Beautiful Soup Library is the bomb
    soup_page = soup(xml_page,"xml")
    news_list = soup_page.findAll("item")
    
    # Two separate lists for News Title and Publication Date
    l1 = []
    l2 = []
    for news in news_list:
        # Append to a list
        l1.append(news.title.text)
        l2.append(news.pubDate.text)
        # Zip the two together
        l_tup = list(zip(l1, l2))
    
    # Save this to a DataFrame
    df = pd.DataFrame(l_tup, columns=['Title', 'Date'])
    # Select Date of Headline
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    # Split the Title into Headline and Source columns and then drop the 'Title' column
    df[['Headline','Source']] = df['Title'].str.rsplit("-",1,expand=True)
    df.drop('Title', axis=1, inplace=True)
    df['Client'] = str(search_query)
    return df

def sentiment_analyser(search_query):
    """
    Runs a Google News Search on the input string and then uses VADER sentiment analysis engine on each returned headline.
    Input: Search Query String
    Output: DataFrame with compound sentiment score for each news article
    """
    # Create a Covid-19 News DataFrame for each organization of interest
    news_df = covid19_news_scraper(search_query)
    # Initialize VADER Sentiment Intensity Analyzer 
    sia = SIA()
    results = []

    # Calculate the polarity score for each headline associated with the organization
    for row in news_df['Headline']:
        pol_score = sia.polarity_scores(row)
        pol_score['Headline'] = row
        results.append(pol_score)
    
    # Create the Sentiment DataFrame
    sent_df = pd.DataFrame.from_records(results)
    # Merge the two dataframes together on the 'Headline' column
    merge_df = news_df.merge(sent_df, on='Headline')
    # Re-order and Rename the columns
    merge_df = merge_df.rename(columns={'compound':'Composite Score'})
    col_order = ['Client','Date','Headline','Source','Composite Score']#,'neg','neu','pos']
    print('Completed processing %s' % search_query, "...")
    return merge_df[col_order]

cba_df = sentiment_analyser('CBA')
print(cba_df.shape)
cba_df.head()

Completed processing CBA ...
(100, 5)


,Client,Date,Headline,Source,Composite Score
0,CBA,2020-04-14,Hoarding boom is over: CBA says shoppers have closed their wallets,The Age,0.0000
1,CBA,2020-04-16,"Sydney, Melbourne house prices facing 10 per cent fall: CBA",Brisbane Times,0.0000
2,CBA,2020-03-31,Coronavirus: CBA report reveals what we’re buying during pandemic,NEWS.com.au,0.0000
3,CBA,2020-03-23,"Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall",Canstar,0.0000
4,CBA,2020-04-01,CBA increases coronavirus loan support,7NEWS.com.au,0.4019


In [9]:
nab_df = sentiment_analyser('NAB')
print(nab_df.shape)
nab_df.head()

Completed processing NAB ...
(100, 5)


,Client,Date,Headline,Source,Composite Score
0,NAB,2020-04-20,"ASX drops 2.5pc as oil prices collapse, while NAB flags $1.1b hit to earnings",ABC News,-0.4939
1,NAB,2020-04-20,NAB flags $1.14bn triple hit to H1 result,The West Australian,0.0000
2,NAB,2020-03-24,NAB worker sacked over false coronavirus test,The New Daily,0.0000
3,NAB,2020-04-17,NAB's McEwan vows fight to save jobs in bank restructure,Sydney Morning Herald,0.1531
4,NAB,2020-04-20,Coronavirus credit card cuts: Which banks have slashed rates?,Mozo.com.au,-0.1280


In [10]:
anz_df = sentiment_analyser('ANZ')
print(anz_df.shape)
anz_df.head()

Completed processing ANZ ...
(100, 5)


,Client,Date,Headline,Source,Composite Score
0,ANZ,2020-03-24,ANZ signals further relief for customers during crisis,6PR,-0.2500
1,ANZ,2020-04-04,'Australia won't look the same': ANZ's Elliott warns coronavirus impact will be generational,The Age,-0.1027
2,ANZ,2020-04-19,'Everybody is paying': ANZ Bank chairman David Gonski,Brisbane Times,0.0000
3,ANZ,2020-03-24,Big bank coronavirus support: How to defer your mortgage repayments,Mozo.com.au,0.1280
4,ANZ,2020-04-19,Gonski flags hit to dividend at ANZ,The Australian,0.0000


In [11]:
wbc_df = sentiment_analyser('Westpac')
print(wbc_df.shape)
wbc_df.head()

Completed processing Westpac ...
(100, 5)


,Client,Date,Headline,Source,Composite Score
0,Westpac,2020-04-16,Riskier funds have been exposed by coronavirus: Westpac,The Australian Financial Review,-0.4019
1,Westpac,2020-04-14,"Westpac flags $1.4b hit to earnings, braces for coronavirus impact",Sydney Morning Herald,0.0000
2,Westpac,2020-04-15,Westpac survey reveals coronavirus caused consumer confidence to fall,NEWS.com.au,0.5106
3,Westpac,2020-04-20,"ASX drops 2.5pc as oil prices collapse, while NAB flags $1.1b hit to earnings",ABC News,-0.4939
4,Westpac,2020-03-23,"Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall",Canstar,0.0000


### Modify the code to work for multiple clients
---

In [12]:
def covid19_news_scraper(search_query):
    """
    Pass in a client name or search query and returns last 100 headlines associating the client with Covid-19   
    """
    # Use this URL for Australian centric data
    news_url = "https://news.google.com.au/rss/search?q={"+str(search_query)+"%coronavirus}"
    Client = urlopen(news_url)
    xml_page = Client.read()
    Client.close()
    # Beautiful Soup Library is the bomb
    soup_page = soup(xml_page,"xml")
    news_list = soup_page.findAll("item")
    
    # Two separate lists for News Title and Publication Date
    l1 = []
    l2 = []
    for news in news_list:
        # Append to a list
        l1.append(news.title.text)
        l2.append(news.pubDate.text)
        # Zip the two together
        l_tup = list(zip(l1, l2))
    
    # Save this to a DataFrame
    df = pd.DataFrame(l_tup, columns=['Title', 'Date'])
    # Select Date of Headline
    df['Date'] = pd.to_datetime(df['Date']).dt.date
    # Split the Title into Headline and Source columns and then drop the 'Title' column
    df[['Headline','Source']] = df['Title'].str.rsplit("-",1,expand=True)
    df.drop('Title', axis=1, inplace=True)
    df['Client'] = str(search_query)
    return df

def sentiment_analyser(search_query):
    """
    Runs a Google News Search on the input string and then uses VADER sentiment analysis engine on each returned headline.
    Input: Search Query String
    Output: DataFrame with compound sentiment score for each news article
    """
    # Create a Covid-19 News DataFrame for each organization of interest
    news_df = covid19_news_scraper(search_query)
    # Initialize VADER Sentiment Intensity Analyzer 
    sia = SIA()
    results = []

    # Calculate the polarity score for each headline associated with the organization
    for row in news_df['Headline']:
        pol_score = sia.polarity_scores(row)
        pol_score['Headline'] = row
        results.append(pol_score)
    
    # Create the Sentiment DataFrame
    sent_df = pd.DataFrame.from_records(results)
    # Merge the two dataframes together on the 'Headline' column
    merge_df = news_df.merge(sent_df, on='Headline')
    # Re-order and Rename the columns
    merge_df = merge_df.rename(columns={'compound':'VADER Score'})
    col_order = ['Client','Date','Headline','Source','VADER Score','neg','neu','pos']
    print('Completed processing %s' % search_query, "...")
    return merge_df[col_order]

def client_c19_news_agg(client_list):
    """
    Provided a list of clients, this pulls up the past 100 covid-19 related news articles on each of them and calculates 
    a Composite Sentiment score for each article related to a client 
    """
    frames = [sentiment_analyser(c) for c in client_list]
    result = pd.concat(frames)
    print()
    print("VADER Score is a Normalized Weighted Sentiment Composite Score that ranges from +1 (Extremely Positive) to -1 (Extremely Negative)")
    return result

clients = ['CBA', 'NAB', 'Westpac', 'ANZ']
df = client_c19_news_agg(clients)
df.sample(10)

Completed processing CBA ...
Completed processing NAB ...
Completed processing Westpac ...
Completed processing ANZ ...

VADER Score is a Normalized Weighted Sentiment Composite Score that ranges from +1 (Extremely Positive) to -1 (Extremely Negative)


,Client,Date,Headline,Source,VADER Score,neg,neu,pos
82,CBA,2020-04-03,Why Commonwealth Bank shares are a no-brainer for income investors,Motley Fool Australia,0.2960,0.000,0.784,0.216
84,CBA,2020-04-04,Banks urged to 'stretch' to help crisis customers,The Age,-0.3400,0.320,0.469,0.211
20,ANZ,2020-04-14,Coronavirus support: What the Big Four banks are offering households and businesses,The New Daily,0.4019,0.000,0.803,0.197
4,Westpac,2020-03-23,"Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall",Canstar,0.0000,0.000,1.000,0.000
44,Westpac,2020-04-09,Westpac supports Government Loan Guarantee Scheme,Mirage News,0.5423,0.000,0.471,0.529
1,CBA,2020-04-16,"Sydney, Melbourne house prices facing 10 per cent fall: CBA",Brisbane Times,0.0000,0.000,1.000,0.000
14,ANZ,2020-03-26,Coronavirus pushes up contactless payment limit to contain virus spread,7NEWS.com.au,0.0000,0.000,1.000,0.000
41,NAB,2020-03-28,NAB offers SMEs new $250000 low-rate loans,Mirage News,0.0000,0.000,1.000,0.000
50,ANZ,2020-04-02,Australian banks face calls to scrap dividends during coronavirus crisis,The New Daily,-0.6249,0.313,0.687,0.000
42,NAB,2020-04-19,"U.S. Lawmakers Make Push to Assist Local Newspapers, Broadcasters",The New York Times,0.0000,0.000,1.000,0.000


In [13]:
df.shape

(400, 8)

### Topic Analysis on Text
---

In [14]:
# Create a spaCy object
nlp = spacy.load('en_core_web_sm')

#### Entity Recognition
---

In [15]:
for client in clients:
    print(df[df.Client==client]['Headline'].head())

0    Hoarding boom is over: CBA says shoppers have closed their wallets 
1    Sydney, Melbourne house prices facing 10 per cent fall: CBA        
2    Coronavirus: CBA report reveals what we’re buying during pandemic  
3    Coronavirus ASX: Coles, BHP Up, CBA, Westpac, NAB, Afterpay Fall   
4    CBA increases coronavirus loan support                             
Name: Headline, dtype: object
0    ASX drops 2.5pc as oil prices collapse, while NAB flags $1.1b hit to earnings 
1    NAB flags $1.14bn triple hit to H1 result                                     
2    NAB worker sacked over false coronavirus test                                 
3    NAB's McEwan vows fight to save jobs in bank restructure                      
4    Coronavirus credit card cuts: Which banks have slashed rates?                 
Name: Headline, dtype: object
0    Riskier funds have been exposed by coronavirus: Westpac                       
1    Westpac flags $1.4b hit to earnings, braces for coronavirus impact

In [16]:
for client in clients:
    for i in range(100):
        doc = nlp(df[df.Client==client]['Headline'][i])
        spacy.displacy.render(doc, style='ent', jupyter=True)

In [17]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

#### Lemmatization
---

In [19]:
for client in clients:
    for i in range(10):
        doc = nlp(df[df.Client==client]['Headline'][i])
        review = str(" ".join([j.lemma_ for j in doc]))
        doc = nlp(review)
        spacy.displacy.render(doc, style='ent',jupyter=True)

In [20]:
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [21]:
tqdm.pandas()
df["Headline"] = df["Headline"].progress_apply(spacy_tokenizer)
df.head()

100%|██████████| 400/400 [00:00<00:00, 1657.07it/s]


,Client,Date,Headline,Source,VADER Score,neg,neu,pos
0,CBA,2020-04-14,hoarding boom cba shopper close wallet,The Age,0.0000,0.0,1.000,0.000
1,CBA,2020-04-16,sydney melbourne house price face 10 cent fall cba,Brisbane Times,0.0000,0.0,1.000,0.000
2,CBA,2020-03-31,coronavirus cba report reveal buy pandemic,NEWS.com.au,0.0000,0.0,1.000,0.000
3,CBA,2020-03-23,coronavirus asx coles bhp cba westpac nab afterpay fall,Canstar,0.0000,0.0,1.000,0.000
4,CBA,2020-04-01,cba increase coronavirus loan support,7NEWS.com.au,0.4019,0.0,0.597,0.403


In [22]:
df.tail()

,Client,Date,Headline,Source,VADER Score,neg,neu,pos
95,ANZ,2020-03-23,australian plenty place turn free financial assistance come month,Daily Telegraph,0.5106,0.0,0.820,0.180
96,ANZ,2020-04-02,commbank offer interest refund home loan credit card customer,Savings.com.au,0.6808,0.0,0.616,0.384
97,ANZ,2020-04-01,australian stock market jump 3.5 cent investor pin hope major bank,9News,0.4215,0.0,0.823,0.177
98,ANZ,2020-04-05,important step home loan customer coronavirus pandemic,PerthNow,0.2023,0.0,0.870,0.130
99,ANZ,2020-04-03,9 place hire worker right,7NEWS.com.au,0.0000,0.0,1.000,0.000


### Topic Modeling with sklearn - to be redone using GENSIM
---

In [24]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english',
                             lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df["Headline"])
NUM_TOPICS = 10

In [25]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [26]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [27]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [28]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [29]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('coronavirus', 12.108495026471733), ('cut', 9.691777020916298), ('rate', 9.467867584769303), ('dividend', 8.31703132216418), ('buy', 8.2906412593394), ('bank', 7.350987947705607), ('spend', 4.515033052400761), ('credit', 4.328008287618024), ('card', 4.319301902767212), ('asx', 3.8753238060334017)]
Topic 1:
[('crisis', 17.912544085768673), ('worker', 15.891386229007685), ('loan', 14.75646489665657), ('customer', 12.567537536462645), ('nab', 11.880317760957944), ('coronavirus', 11.327298154182317), ('credit', 10.800468250671095), ('card', 10.749134054743365), ('home', 10.56127183100596), ('offer', 10.464030505740533)]
Topic 2:
[('coronavirus', 26.730100748468928), ('pandemic', 19.62417912777104), ('home', 10.410343475253306), ('loan', 10.319935300544417), ('job', 9.920832428539269), ('customer', 8.689824171649104), ('relief', 7.806980415282623), ('hit', 6.1603713667008275), ('reveal', 5.77937928566801), ('million', 5.115857519621089)]
Topic 3:
[('bank', 25.330438152

In [30]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('coronavirus', 4.716631291649473), ('crisis', 0.4602247605758719), ('australia', 0.3883804996688655), ('pandemic', 0.2767452569078068), ('impact', 0.1910020102662019), ('job', 0.18715867856535462), ('push', 0.15451522326062977), ('spread', 0.1421902751359621), ('limit', 0.1291827339391156), ('payment', 0.12734347403907278)]
Topic 1:
[('bank', 3.3111000923531275), ('mortgage', 0.5244241624572922), ('commonwealth', 0.4075055501015735), ('branch', 0.3866712266332575), ('hold', 0.3714071642259443), ('amid', 0.36546115307722676), ('close', 0.3542282649246589), ('freeze', 0.33800595964550184), ('payment', 0.28855807174060233), ('australian', 0.20983251834066496)]
Topic 2:
[('nab', 3.69867366207072), ('branch', 0.34541373744897397), ('flag', 0.27173816720180805), ('close', 0.2693354587914116), ('collapse', 0.2544582305076172), ('low', 0.21475336375890328), ('commonwealth', 0.1473793882953752), ('consumer', 0.14273391971702853), ('oil', 0.14245888943432142), ('worker', 0.

In [31]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('coronavirus', 0.8177223413434412), ('bank', 0.37685601691614484), ('australian', 0.13530716993321396), ('business', 0.11794244523238191), ('nab', 0.11357574304847509), ('asx', 0.10011495434283109), ('westpac', 0.09189683618615513), ('loan', 0.08374972064393095), ('crisis', 0.08229721254911022), ('big', 0.08148087225333067)]
Topic 1:
[('bank', 0.7579592833411971), ('big', 0.2074186409429326), ('business', 0.1516377345353304), ('dividend', 0.12301603292445024), ('offer', 0.11549376650075129), ('mortgage', 0.11113105364385024), ('loan', 0.10432004018498592), ('support', 0.10180817418037655), ('commonwealth', 0.09079039289628142), ('hold', 0.07870549215379484)]
Topic 2:
[('asx', 0.47852646893201184), ('nab', 0.4248689893124374), ('westpac', 0.2880763440141423), ('dividend', 0.1984529657921083), ('hit', 0.17952462867900756), ('share', 0.15551583522016152), ('buy', 0.13703797455228325), ('fall', 0.1319743472268919), ('cba', 0.13126111780257665), ('price', 0.12776017350

In [32]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6     -22.933094   265.509949  1       1        15.866421
5     -17.134909  -219.466309  2       1        15.089588
9     -10.483713   22.306877   3       1        15.021615
3      173.475311  179.167023  4       1        11.393890
2      166.867081 -62.496277   5       1        9.398098 
1      179.463684 -305.536346  6       1        9.095799 
0      365.608582 -157.376587  7       1        8.232820 
7     -207.556198 -120.479050  8       1        6.798704 
4     -209.109818  117.429886  9       1        6.580563 
8      363.989838  80.499840   10      1        2.522501 , topic_info=    Category        Freq          Term       Total  loglift  logprob
5    Default  47.000000   australian    47.000000   30.0000  30.0000
6    Default  71.000000   bank          71.000000   29.0000  29.0000
59   Default  39.000000   loan          39.000000   28.0000  28.0000
68   Default  13.000000   month         13.000000   27.0000  27.0000
99   Default  25.000000   support       25.000000   26.0000  26.0000
22   Default  186.000000  coronavirus   186.000000  25.0000  25.0000
41   Default  13.000000   financial     13.000000   24.0000  24.0000
1    Default  37.000000   anz           37.000000   23.0000  23.0000
77   Default  21.000000   pandemic      21.000000   22.0000  22.0000
38   Default  17.000000   face          17.000000   21.0000  21.0000
11   Default  28.000000   business      28.000000   20.0000  20.0000
25   Default  26.000000   crisis        26.000000   19.0000  19.0000
103  Default  15.000000   update        15.000000   18.0000  18.0000
16   Default  19.000000   close         19.000000   17.0000  17.0000
4    Default  24.000000   australia     24.000000   16.0000  16.0000
10   Default  19.000000   branch        19.000000   15.0000  15.0000
51   Default  26.000000   home          26.000000   14.0000  14.0000
23   Default  27.000000   covid-        27.000000   13.0000  13.0000
27   Default  14.000000   cut           14.000000   12.0000  12.0000
14   Default  25.000000   cba           25.000000   11.0000  11.0000
26   Default  19.000000   customer      19.000000   10.0000  10.0000
86   Default  15.000000   rate          15.000000   9.0000   9.0000 
2    Default  34.000000   asx           34.000000   8.0000   8.0000 
31   Default  23.000000   dividend      23.000000   7.0000   7.0000 
109  Default  13.000000   worker        13.000000   6.0000   6.0000 
39   Default  19.000000   fall          19.000000   5.0000   5.0000 
12   Default  14.000000   buy           14.000000   4.0000   4.0000 
13   Default  18.000000   card          18.000000   3.0000   3.0000 
24   Default  16.000000   credit        16.000000   2.0000   2.0000 
108  Default  29.000000   westpac       29.000000   1.0000   1.0000 
39   Topic1   18.440913   fall          19.641001   1.7779  -2.6241 
21   Topic1   12.027740   consumer      13.135973   1.7528  -3.0515 
20   Topic1   11.996671   confidence    13.192510   1.7459  -3.0540 
82   Topic1   9.699656    price         10.984877   1.7165  -3.2666 
42   Topic1   6.038283    flag          7.122234    1.6759  -3.7405 
17   Topic1   4.454116    collapse      5.657297    1.6018  -4.0448 
23   Topic1   21.173240   covid-        27.049664   1.5960  -2.4859 
91   Topic1   8.928560    rise          13.037997   1.4624  -3.3494 
74   Topic1   3.927327    oil           5.767987    1.4566  -4.1707 
80   Topic1   3.340193    plunge        4.946840    1.4482  -4.3326 
34   Topic1   5.041431    drop          7.584158    1.4326  -3.9210 
62   Topic1   8.872787    low           13.833431   1.3969  -3.3557 
108  Topic1   17.671689   westpac       29.012898   1.3452  -2.6667 
14   Topic1   14.379353   cba           25.860312   1.2540  -2.8729 
93   Topic1   9.167451    share         16.620494   1.2460  -3.3230 
56   Topic1   2.390519    jones         4.843722    1.1348  -4.6672 
32   Topic1   2.38